In [10]:
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch([{'host':'uct2-es-door.mwt2.org', 'port':9200}])

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
plt.style.use('ggplot')

In [11]:
ind="stashcp-2016.7"

myquery1={
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "query": "*",
          "analyze_wildcard": True
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "query": {
                "query_string": {
                  "analyze_wildcard": True,
                  "query": "*"
                }
              }
            },
            {
              "range": {
                "timestamp": {
                  "gte": 1467433769609,
                  "lte": 1467437864765,
                  "format": "epoch_millis"
                }
              }
            }
          ],
        }
      }
    }
  },
}

#nebraska
myquery2={
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "query": "*",
          "analyze_wildcard": True
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "query": {
                "query_string": {
                  "analyze_wildcard": True,
                  "query": "*"
                }
              }
            },
            {
              "range": {
                "timestamp": {
                  "gte": 1467650828067,
                  "lte": 1467656645210,
                  "format": "epoch_millis"
                }
              }
            }
          ],
        }
      }
    }
  },
}

In [28]:
page= es.search(index=ind, body=myquery2, scroll='2m', search_type='scan', size=1)

sid = page['_scroll_id']

scroll_size = page['hits']['total']
results=[]

while (scroll_size > 0):
    page = es.scroll(scroll_id = sid, scroll = '2m')
    results.append(page['hits']['hits'])
    sid = page['_scroll_id']
    scroll_size = len(page['hits']['hits'])

scroll_size = page['hits']['total']
Res=[]
for i in range(0,scroll_size):
    Res.append(results[i][0]['_source'])
    
print(scroll_size)

df = pd.DataFrame(Res)
#print(df.head(2))

del df['destination_space']
del df['xrdcp_version']
#del df['tries']
del df['download_size']
del df['filesize']
del df['IP']
del df['xrdcp_exit']

begin=[]
for i in range(scroll_size):
    begin.append((int(df['timestamp'][i])-int(df['download_time'][i]))) #convert to minutes
    
df['begin']=begin
print(list(df.columns.values))
print(df.head(1))

993
['cache', 'download_time', 'end1', 'end2', 'filename', 'host', 'sitename', 'start1', 'start2', 'start3', 'status', 'timestamp', 'tries', 'begin']
                                               cache download_time  \
0  root://mwt2-stashcache.campuscluster.illinois.edu         60066   

            end1 end2                           filename  \
0  1467651270056    0  user/taburaad/public/2gb_file.tar   

                                                host sitename         start1  \
0  root://mwt2-stashcache.campuscluster.illinois.edu   osg-ce  1467651209990   

  start2 start3   status      timestamp tries          begin  
0      0      0  Success  1467651270000     1  1467651209934  


In [29]:
length=df.shape[0]

for i in range(length):
    begin=int(df['begin'].iloc[i])//1000
    end=int(df['timestamp'].iloc[i])//1000
    start1=int(df['start1'].iloc[i])//1000
    end1=int(df['end1'].iloc[i])//1000
    start2=int(df['start2'].iloc[i])//1000
    end2=int(df['end2'].iloc[i])//1000
    start3=int(df['start3'].iloc[i])//1000
    df['begin'].iloc[i]=datetime.datetime.fromtimestamp(begin).strftime('%Y-%m-%d %H:%M:%S')
    df['timestamp'].iloc[i]=datetime.datetime.fromtimestamp(end).strftime('%Y-%m-%d %H:%M:%S')  
    df['start1'].iloc[i]=datetime.datetime.fromtimestamp(start1).strftime('%Y-%m-%d %H:%M:%S')
    df['start2'].iloc[i]=datetime.datetime.fromtimestamp(start2).strftime('%Y-%m-%d %H:%M:%S')
    df['start3'].iloc[i]=datetime.datetime.fromtimestamp(start3).strftime('%Y-%m-%d %H:%M:%S')
    df['end1'].iloc[i]=datetime.datetime.fromtimestamp(end1).strftime('%Y-%m-%d %H:%M:%S')
    df['end2'].iloc[i]=datetime.datetime.fromtimestamp(end2).strftime('%Y-%m-%d %H:%M:%S')


    
df=df.set_index(['timestamp'])    
df.index = pd.to_datetime(df.index, unit='s')
df['begin']=pd.to_datetime(df.begin, unit='s')
df['start1']=pd.to_datetime(df.begin, unit='s')
df['start2']=pd.to_datetime(df.begin, unit='s')
df['start3']=pd.to_datetime(df.begin, unit='s')
df['end1']=pd.to_datetime(df.begin, unit='s')
df['end2']=pd.to_datetime(df.begin, unit='s')
df

/home/ivukotic/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,cache,download_time,end1,end2,filename,host,sitename,start1,start2,start3,status,tries,begin
timestamp,,,,,,,,,,,,,
2016-07-04 11:54:30,root://mwt2-stashcache.campuscluster.illinois.edu,60066,2016-07-04 11:53:29,2016-07-04 11:53:29,user/taburaad/public/2gb_file.tar,root://mwt2-stashcache.campuscluster.illinois.edu,osg-ce,2016-07-04 11:53:29,2016-07-04 11:53:29,2016-07-04 11:53:29,Success,1,2016-07-04 11:53:29
2016-07-04 11:54:30,root://xrd-cache-1.t2.ucsd.edu,120281,2016-07-04 11:52:29,2016-07-04 11:52:29,user/taburaad/public/2gb_file.tar,root://xrd-cache-1.t2.ucsd.edu,CIT_CMS_T2B,2016-07-04 11:52:29,2016-07-04 11:52:29,2016-07-04 11:52:29,Success,1,2016-07-04 11:52:29
2016-07-04 11:54:31,root://data.ci-connect.net,60074,2016-07-04 11:53:30,2016-07-04 11:53:30,user/taburaad/public/2gb_file.tar,root://data.ci-connect.net,MWT2_CE_UIUC,2016-07-04 11:53:30,2016-07-04 11:53:30,2016-07-04 11:53:30,Success,1,2016-07-04 11:53:30
2016-07-04 11:54:31,root://hcc-stash.unl.edu,60136,2016-07-04 11:53:30,2016-07-04 11:53:30,user/taburaad/public/2gb_file.tar,root://hcc-stash.unl.edu,Sandhills-CE1,2016-07-04 11:53:30,2016-07-04 11:53:30,2016-07-04 11:53:30,Success,1,2016-07-04 11:53:30
2016-07-04 11:54:31,root://hcc-stash.unl.edu,60162,2016-07-04 11:53:30,2016-07-04 11:53:30,user/taburaad/public/2gb_file.tar,root://hcc-stash.unl.edu,Sandhills-CE1,2016-07-04 11:53:30,2016-07-04 11:53:30,2016-07-04 11:53:30,Success,1,2016-07-04 11:53:30
2016-07-04 11:54:32,root://hcc-stash.unl.edu,60047,2016-07-04 11:53:31,2016-07-04 11:53:31,user/taburaad/public/2gb_file.tar,root://hcc-stash.unl.edu,Crane-CE1,2016-07-04 11:53:31,2016-07-04 11:53:31,2016-07-04 11:53:31,Success,1,2016-07-04 11:53:31
2016-07-04 11:54:32,root://hcc-stash.unl.edu,60019,2016-07-04 11:53:31,2016-07-04 11:53:31,user/taburaad/public/2gb_file.tar,root://hcc-stash.unl.edu,Red,2016-07-04 11:53:31,2016-07-04 11:53:31,2016-07-04 11:53:31,Success,1,2016-07-04 11:53:31
2016-07-04 11:54:32,root://hcc-stash.unl.edu,60046,2016-07-04 11:53:31,2016-07-04 11:53:31,user/taburaad/public/2gb_file.tar,root://hcc-stash.unl.edu,Crane-CE1,2016-07-04 11:53:31,2016-07-04 11:53:31,2016-07-04 11:53:31,Success,1,2016-07-04 11:53:31
2016-07-04 11:54:32,root://hcc-stash.unl.edu,60040,2016-07-04 11:53:31,2016-07-04 11:53:31,user/taburaad/public/2gb_file.tar,root://hcc-stash.unl.edu,Crane-CE1,2016-07-04 11:53:31,2016-07-04 11:53:31,2016-07-04 11:53:31,Success,1,2016-07-04 11:53:31


In [30]:
#df.to_pickle('nebraskadata1.pkl') 

type(df['start1'].iloc[1])

pandas.tslib.Timestamp

In [32]:
jobs = pd.concat([pd.Series(1, df.begin), pd.Series(-1, df.index)]).resample('1Min', how='sum').cumsum()
index=jobs.index

df2=pd.DataFrame(columns=['index', 'Origin', 'Nebraska', 'UCSD', 'US-MWT2_UIUC', 'BNL ATLAS', 'FZU', 'Trunk'])
df2['index']=index

length2=df2.shape[0]
df2['Origin']=0
df2['Nebraska']=0
df2['UCSD']=0
df2['US-MWT2_UIUC']=0
df2['BNL ATLAS']=0
df2['FZU']=0
df2['Trunk']=0


for i in range(length2):
    for j in range(length):
        if df['tries'].iloc[j]==1:
            if df['begin'].iloc[j]<=df2['index'].iloc[i] and df.index[j]>=df2['index'].iloc[i]:
                if df['host'].iloc[j]=='root://data.ci-connect.net':
                    df2['Origin'].iloc[i]+=1
                if df['host'].iloc[j]=='root://hcc-stash.unl.edu':
                    df2['Nebraska'].iloc[i]+=1
                if df['host'].iloc[j]=='root://mwt2-stashcache.campuscluster.illinois.edu':
                    df2['US-MWT2_UIUC'].iloc[i]+=1
                if df['host'].iloc[j]=='root://xrd-cache-1.t2.ucsd.edu':
                    df2['UCSD'].iloc[i]+=1
                if df['host'].iloc[j]=='root://osgxroot.usatlas.bnl.gov':
                    df2['BNL ATLAS'].iloc[i]+=1
                if df['host'].iloc[j]=='novastore.farm.particle.cz':
                    df2['FZU'].iloc[i]+=1
        elif df['tries'].iloc[j]!=1:
            if df['start2'].iloc[j]<=df2['index'].iloc[i] and df.index[j]>=df2['index'].iloc[i]:
                if df['cache'].iloc[j]=='root://hcc-stash.unl.edu':
                    df2['Nebraska'].iloc[i]+=1
                if df['cache'].iloc[j]=='root://mwt2-stashcache.campuscluster.illinois.edu':
                    df2['US-MWT2_UIUC'].iloc[i]+=1
                if df['cache'].iloc[j]=='root://xrd-cache-1.t2.ucsd.edu':
                    df2['UCSD'].iloc[i]+=1
                if df['cache'].iloc[j]=='root://osgxroot.usatlas.bnl.gov':
                    df2['BNL ATLAS'].iloc[i]+=1
                if df['cache'].iloc[j]=='novastore.farm.particle.cz':
                    df2['FZU'].iloc[i]+=1
                if df['cache'].iloc[j]=='root://data.ci-connect.net':
                    df2['Trunk'].iloc[i]+=1




df2



/home/ivukotic/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  if __name__ == '__main__':
/home/ivukotic/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,index,Origin,Nebraska,UCSD,US-MWT2_UIUC,BNL ATLAS,FZU,Trunk
0,2016-07-04 11:50:00,0,0,0,0,0,0,0
1,2016-07-04 11:51:00,0,0,0,0,1,0,0
2,2016-07-04 11:52:00,0,0,0,0,3,0,0
3,2016-07-04 11:53:00,0,24,29,7,11,0,19
4,2016-07-04 11:54:00,0,27,29,14,30,0,24
5,2016-07-04 11:55:00,0,27,60,15,256,0,21
6,2016-07-04 11:56:00,0,27,61,15,323,0,18
7,2016-07-04 11:57:00,0,39,75,29,332,0,27
8,2016-07-04 11:58:00,0,26,75,27,343,0,27
9,2016-07-04 11:59:00,0,6,126,11,442,0,10


In [33]:
#df2

In [ ]:
df2.plot.bar(stacked=True,figsize=[24, 12])
plt.title('Number of jobs running on each server per minute over a submit of 1,000 jobs directed to UCSDT2 (400 shown)', fontsize=20) 
plt.xlabel('Minutes', fontsize=15)
plt.ylabel('Count of concurrent jobs',fontsize=15)
#plt.text(60,55,'*No GLIDEIN_ResourceName specified in condor.submit')
#plt.savefig('Ilija_plot1_2.png')


In [24]:
3!=2

True